In [3]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        path = os.path.join(folder, label)
        if os.path.isdir(path):
            for image_filename in os.listdir(path):
                img = cv2.imread(os.path.join(path, image_filename), cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    images.append(img)
                    labels.append(label)
    return images, labels

folder = "data/15-Scene"
images, labels = load_images_from_folder(folder)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.3, random_state=42)



def extract_sift_features(X):
    sift = cv2.SIFT_create()
    descriptors = []
    for img in X:
        kp, des = sift.detectAndCompute(img, None)
        descriptors.append(des)
    return descriptors

train_descriptors = extract_sift_features(X_train)
test_descriptors = extract_sift_features(X_test)



def build_vocabulary(descriptors_list, k):
    all_descriptors = np.vstack(descriptors_list)
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(all_descriptors)
    return kmeans

k = 100  # 根据需要调整K值
vocab_model = build_vocabulary(train_descriptors, k)


def features_to_histogram(features, vocab_model):
    histogram = np.zeros(len(vocab_model.cluster_centers_))
    words = vocab_model.predict(features)
    for w in words:
        histogram[w] += 1
    return histogram

def prepare_histograms(descriptors_list, vocab_model):
    histograms = []
    for descriptors in descriptors_list:
        if descriptors is not None:
            histogram = features_to_histogram(descriptors, vocab_model)
            histograms.append(histogram)
        else:
            histograms.append(None)
    return histograms

train_histograms = prepare_histograms(train_descriptors, vocab_model)
test_histograms = prepare_histograms(test_descriptors, vocab_model)


# 处理可能的None值
train_histograms = [hist if hist is not None else np.zeros(k) for hist in train_histograms]
test_histograms = [hist if hist is not None else np.zeros(k) for hist in test_histograms]

# SVM训练
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

clf = SVC(kernel='linear')  # 可以尝试其他核
clf.fit(train_histograms, y_train_encoded)


predictions = clf.predict(test_histograms)
print(classification_report(y_test_encoded, predictions, target_names=le.classes_))


d:\Project\CodeProject\nlp\LLM\chat_venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


              precision    recall  f1-score   support

          00       0.33      0.38      0.35        71
          01       0.86      0.85      0.86        80
          02       0.20      0.31      0.25        86
          03       0.26      0.36      0.30        58
          04       0.37      0.31      0.34       106
          05       0.58      0.76      0.66       115
          06       0.73      0.81      0.77        88
          07       0.53      0.58      0.56        72
          08       0.54      0.42      0.47        98
          09       0.58      0.56      0.57       101
          10       0.48      0.43      0.45       114
          11       0.52      0.39      0.45        84
          12       0.59      0.45      0.51       109
          13       0.37      0.33      0.35        57
          14       0.59      0.45      0.51       107

    accuracy                           0.50      1346
   macro avg       0.50      0.49      0.49      1346
weighted avg       0.51   